In [1]:
%load_ext cython

In [2]:
import numpy as np

In [3]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double, reshape
from libc.math cimport sqrt

cdef void mean_1d(const double[:] x, double* mean):
    cdef Py_ssize_t n = x.size, i

    for i in range(n):
        mean[0] += x[i]
    mean[0] /= n


cpdef covariance(const double[:] x, const double[:] y, int ddof):
    cdef Py_ssize_t n = x.size, i

    cdef double kx = x[0], ky = y[0]
    cdef double Ex = 0., Ey = 0., Exy = 0., cov

    for i in range(n):
        Ex += (x[i] - kx)
        Ey += (y[i] - ky)
        Exy += (x[i] - kx) * (y[i] - ky)

    cov = (Exy - (Ex * Ey) / (n - ddof)) / (n - ddof)
    return cov


cpdef linregress(const double[:] x, const double[:] y):
    cdef Py_ssize_t n = x.size
    if n != y.size:
        raise ValueError('Inputs must be the same size.')
    if n < 2:
        raise ValueError('Inputs must have more than 1 element.')
    
    cdef double xmean, ymean, ssxm, ssxym, ssym
    cdef double slope, intercept

    mean_1d(x, &xmean)
    mean_1d(y, &ymean)

    # average sum of squares
    ssxm = covariance(x, x, 0)
    ssym = covariance(y, y, 0)
    ssxym = covariance(x, y, 0)

    slope = ssxym / ssxm
    intercept = ymean - slope * xmean

    return slope, intercept


cpdef LinRegression(const double[:] x, const double[:, :, :] y):
    cdef Py_ssize_t m = y.shape[0], p = y.shape[2], i, k

    slp = zeros((m, p), dtype=npy_double)
    itcpt = zeros((m, p), dtype=npy_double)
    cdef double[:, :] slope = slp, intercept = itcpt
    cdef double[:, :]  x2d

    for i in range(m):
        for k in range(p):
            slope[i, k], intercept[i, k] = linregress(x, y[i, :, k])
    
    return slp, itcpt

In [4]:
from linearslope import linearregression as flinreg

In [5]:
x = np.random.rand(50000, 150, 3)
xf = np.asfortranarray(x.transpose([1, 2, 0]))

t = np.arange(0, 150/50, 1/50)

In [6]:
%timeit LinRegression(t, x)

362 ms ± 9.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit flinreg(t, xf)

25.8 ms ± 82.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
np.allclose(flinreg(t, xf).T, LinRegression(t, x)[0])

True

In [ ]:
flinreg(t, xf).T

In [ ]:
LinRegression(t, x)[0]

In [ ]:
from linearslope import covariance as fcov

In [ ]:
print(fcov(t, x[0, :, 0], 0))
print(covariance(t, x[0, :, 0], 0))